In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd /content/gdrive/MyDrive/AdvML/

/content/gdrive/MyDrive/AdvML


In [ ]:
import torch

if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

In [ ]:
batch_size = 100

In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataset = torchvision.datasets.MNIST(root='.', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='.', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def de_norm(image):
  return image * 0.5 + 0.5

In [ ]:
train_dataset[0][0].shape

torch.Size([1, 28, 28])

In [ ]:
wh = 28 * 28

In [ ]:
import torch.nn as nn

D = nn.Sequential(
    nn.Linear(wh, 128),
    nn.LeakyReLU(0.01),
    nn.Linear(128, 1),
    nn.Sigmoid()
)

D.to(device)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=128, out_features=1, bias=True)
  (3): Sigmoid()
)

In [ ]:
d_criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)

In [ ]:
z_dim = 100

In [ ]:
G = nn.Sequential(
    nn.Linear(z_dim, 128),
    nn.LeakyReLU(0.01),
    nn.Linear(128, wh),
    nn.Tanh()
)

G.to(device)

Sequential(
  (0): Linear(in_features=100, out_features=128, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=128, out_features=784, bias=True)
  (3): Tanh()
)

In [ ]:
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [ ]:
def train_discriminator(images):
  images = images.to(device)
  y = torch.ones(batch_size, 1).to(device)
  y_hat = D(images)
  d_real_loss = d_criterion(y_hat, y)
  real_score = y_hat

  z = torch.randn(batch_size, z_dim).to(device)
  fake_images = G(z)
  fake_images = fake_images.to(device)
  y = torch.zeros(batch_size, 1).to(device)
  y_hat = D(fake_images)
  d_fake_loss = d_criterion(y_hat, y)
  fake_score = y_hat

  d_loss = d_real_loss + d_fake_loss
  d_optimizer.zero_grad()
  g_optimizer.zero_grad()
  d_loss.backward()
  d_optimizer.step()

  return d_loss.item(), real_score, fake_score

In [ ]:
def train_generator():
  z = torch.randn(batch_size, z_dim).to(device)
  fake_images = G(z)
  fake_images = fake_images.to(device)
  y = torch.ones(batch_size, 1).to(device)
  y_hat = D(fake_images)
  g_loss = d_criterion(y_hat, y)

  d_optimizer.zero_grad()
  g_optimizer.zero_grad()
  g_loss.backward()
  g_optimizer.step()

  return g_loss.item()

In [ ]:
num_epochs = 300

In [ ]:
for epoch in range(num_epochs):
  for i, (images, _) in enumerate(train_loader):
    images = images.reshape(batch_size, -1)

    d_loss, real_score, fake_score = train_discriminator(images)
    g_loss = train_generator()

  with torch.no_grad():
    print(f'epoch [{epoch + 1}/{num_epochs}], step [{i + 1}/{len(train_loader)}], d_loss: {d_loss:.2f}, g_loss: {g_loss:.2f}, real_score: {real_score.mean().item():.2f}, fake_score: {fake_score.mean().item():.2f}')

epoch [1/300], step [600/600], d_loss: 1.05, g_loss: 0.81, real_score: 0.72, fake_score: 0.51
epoch [2/300], step [600/600], d_loss: 0.63, g_loss: 1.52, real_score: 0.75, fake_score: 0.28
epoch [3/300], step [600/600], d_loss: 1.38, g_loss: 0.80, real_score: 0.57, fake_score: 0.54
epoch [4/300], step [600/600], d_loss: 1.00, g_loss: 1.01, real_score: 0.62, fake_score: 0.39
epoch [5/300], step [600/600], d_loss: 1.05, g_loss: 1.01, real_score: 0.62, fake_score: 0.41
epoch [6/300], step [600/600], d_loss: 1.35, g_loss: 0.73, real_score: 0.54, fake_score: 0.50
epoch [7/300], step [600/600], d_loss: 1.11, g_loss: 0.96, real_score: 0.63, fake_score: 0.43
epoch [8/300], step [600/600], d_loss: 1.37, g_loss: 0.83, real_score: 0.55, fake_score: 0.50
epoch [9/300], step [600/600], d_loss: 0.70, g_loss: 1.38, real_score: 0.71, fake_score: 0.27
epoch [10/300], step [600/600], d_loss: 0.88, g_loss: 1.19, real_score: 0.68, fake_score: 0.36
epoch [11/300], step [600/600], d_loss: 1.38, g_loss: 0.82,

In [ ]:
C = nn.Sequential(
    nn.Conv2d(1, 36, kernel_size=5, stride=1, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(36, 32, kernel_size=5, stride=1, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Linear(32 * 7 * 7, 10)
)

C.to(device)

In [ ]:
c_criterion = nn.CrossEntropyLoss()
c_optimizer = torch.optim.Adam(C.parameters(), lr=0.01)

In [ ]:
num_epochs = 3

In [ ]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)
    y_hat = C(images)
    c_loss = c_criterion(y_hat, labels)

    c_optimizer.zero_grad()
    c_loss.backward()
    c_optimizer.step()

  y_pred = torch.max(y_hat, 1)[1].data.squeeze()
  accuracy = (labels == y_pred).sum().item() / float(labels.size(0))

  print(f'epoch [{epoch + 1}/{num_epochs}], step [{i + 1}/{len(train_loader)}], c_loss: {c_loss.item():.2f}, accuracy_score: {accuracy:.2f}')

In [ ]:
import pickle

with open('D.pkl', 'wb') as f:
  pickle.dump(D, f)

with open('G.pkl', 'wb') as f:
  pickle.dump(G, f)

with open('C.pkl', 'wb') as f:
  pickle.dump(C, f)

In [ ]:
import os
from torchvision.utils import save_image

bs = 100

z = torch.randn(bs, z_dim).to(device)
fake_images = G(z).reshape(bs, 1, 28, 28)

for i, (fake_image, zi) in enumerate(zip(fake_images, z)):
  f_name = str(i + 1).zfill(3)

  save_image(de_norm(fake_image), os.path.join('Fake_Digits', f'{f_name}.png'))

  with open(os.path.join('Fake_Digits', f'{f_name}.txt'), 'w') as f:
    f.write(str(zi.tolist()))

In [ ]:
from torchvision import datasets

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

S1 = datasets.ImageFolder(root='Fake_Digits', transform=transform)

for i in range(len(S1)):
  img_path, label = S1.imgs[i]
  folder_name = os.path.basename(os.path.dirname(img_path))
  S1.samples[i] = (img_path, int(folder_name))

s1_loader = DataLoader(dataset=S1, batch_size=batch_size, shuffle=False)

In [ ]:
for dl in [test_loader, s1_loader]:
  with torch.no_grad():
    n_samples, n_correct = 0, 0

    for images, labels in dl:
      images = images.to(device)
      labels = labels.to(device)
      y_hat = C(images)
      y_pred = torch.max(y_hat, 1)[1].data.squeeze()

      n_samples += labels.size(0)
      n_correct += (labels == y_pred).sum().item()

      break

    accuracy = n_correct / n_samples
    print(accuracy)

0.95
1.0
